In [1]:
import pickle
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import trim_mean

In [2]:
#load dictionary of masks (pixel locations of correctly identified towers)
with open("masks_dict.pkl", "rb") as md:
    final_mask_dict = pickle.load(md)

In [3]:
final_mask_dict.keys()

dict_keys(['frame11288_splash.jpg', 'frame11289_splash.jpg', 'frame12048_splash.jpg', 'frame12049_splash.jpg', 'frame12050_splash.jpg', 'frame12052_splash.jpg', 'frame12053_splash.jpg', 'frame12054_splash.jpg', 'frame12055_splash.jpg', 'frame12056_splash.jpg', 'frame12057_splash.jpg', 'frame12058_splash.jpg', 'frame12349_splash.jpg', 'frame12350_splash.jpg', 'frame12351_splash.jpg', 'frame12669_splash.jpg', 'frame12670_splash.jpg', 'frame12672_splash.jpg', 'frame13731_splash.jpg', 'frame13732_splash.jpg', 'frame14431_splash.jpg', 'frame16433_splash.jpg', 'frame16434_splash.jpg', 'frame18135_splash.jpg', 'frame33152_splash.jpg', 'frame7920_splash.jpg', 'frame7973_splash.jpg'])

In [4]:
depth_vls = {}
for img_file in final_mask_dict.keys():
    #remove one incorrect mask from one image
    if img_file == 'frame12048_splash.jpg':
        final_mask_dict[img_file] = np.delete(final_mask_dict[img_file],1,2)
    #identify locations (indices) of pixels where splash is located
    splash_locs = np.argwhere(final_mask_dict[img_file][:,:,0]==True)
    #run predict.py depth prediction code for that image
    img_path = "~/Documents/GitHub/CV-to-Maps/Mask_RCNN/samples/water_towers/splash/" + img_file
    %run '../../../../FCRN-DepthPrediction/tensorflow/predict_nomain.py' \
    '../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt' $img_path 
    time.sleep(5)
    #load depth prediction array for one image
    with open("../../../../FCRN-DepthPrediction/tensorflow/pred_array.pkl", "rb") as pa:
        depth_array = pickle.load(pa)[0,:,:,0]
    #enlarge the array to match the size of the mask array
    resized_depth_arr = cv2.resize(depth_array, (1920,1080), interpolation = cv2.INTER_CUBIC)
    #access the pixels of mask in the depth prediction array and compute 10% trimmed mean over those depth values
    #depth_vls[img_file] = trim_mean(resized_depth_arr[splash_locs[:,0],splash_locs[:,1]],0.1)
    depth_vls[img_file] = trim_mean(resized_depth_arr[tuple(splash_locs.T)],0.1)
    print(img_file, " processing completed")

Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt


/Users/sashakapralov/anaconda3/envs/py36_2/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


frame11288_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame11289_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame12048_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame12049_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame12050_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame12052_splash.jpg  processing completed
Loading the model
INFO:tensorflow:Restoring parameters from ../../../../FCRN-DepthPrediction/tensorflow/NYU_FCRN.ckpt
frame12053_splash.jpg  proce

In [5]:
depth_vls

{'frame11288_splash.jpg': 1.1417814,
 'frame11289_splash.jpg': 1.0169624,
 'frame12048_splash.jpg': 0.9858812,
 'frame12049_splash.jpg': 0.96916014,
 'frame12050_splash.jpg': 0.9155358,
 'frame12052_splash.jpg': 0.91025686,
 'frame12053_splash.jpg': 0.9102273,
 'frame12054_splash.jpg': 0.9096894,
 'frame12055_splash.jpg': 0.9093233,
 'frame12056_splash.jpg': 0.9055825,
 'frame12057_splash.jpg': 0.89271754,
 'frame12058_splash.jpg': 1.2999741,
 'frame12349_splash.jpg': 1.0757191,
 'frame12350_splash.jpg': 0.9950051,
 'frame12351_splash.jpg': 1.2459457,
 'frame12669_splash.jpg': 2.1056335,
 'frame12670_splash.jpg': 2.0850503,
 'frame12672_splash.jpg': 0.9919193,
 'frame13731_splash.jpg': 1.6799518,
 'frame13732_splash.jpg': 0.95637995,
 'frame14431_splash.jpg': 0.84524286,
 'frame16433_splash.jpg': 1.5303446,
 'frame16434_splash.jpg': 1.3227637,
 'frame18135_splash.jpg': 1.0443932,
 'frame33152_splash.jpg': 1.2270098,
 'frame7920_splash.jpg': 1.3969398,
 'frame7973_splash.jpg': 0.9277557

In [7]:
#save depth values dictionary to pickle file
with open("depth_values.pkl", "wb") as dv:
    pickle.dump(depth_vls,dv)